In [ ]:
import json
import itertools
import xgi
import numpy as np
import networkx as nx
from tqdm import tqdm
import pickle as pk

dataset='congress-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-committees_simplices';name_file='{0}'.format(dataset);
#dataset='house-committees_simplices';name_file='{0}'.format(dataset);
#dataset='email-Enron_simplices'; name_file='{0}'.format(dataset); 
#dataset='email-Eu_simplices'; name_file='{0}'.format(dataset); 
#dataset='hyperedges-cat-edge-algebra-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-geometry-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-music-blues-reviews'; name_file='{0}'.format(dataset);
#dataset='Mid1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Elem1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='InVS15'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='SFHH'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LH10'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LyonSchool'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Thiers13'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='M_PL_015_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_062_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);

with open('Data\{0}.json'.format(name_file)) as json_file:
    data = json.load(json_file)

data = [data[i] for i in range(len(data)) if len(data[i])>1] #select only interactions of size >=2
for i in range(len(data)):
    data[i].sort() #order nodes in interactions according to their ID, e.g. [2,1] -> [1,2]
data.sort()
data = list(data for data,_ in itertools.groupby(data)) #delete duplicate interactions
data.sort(key = len)

L=len(data) 
G = xgi.Hypergraph(data) # build the hypergraph
N=len(G.nodes)

idx_orig=list(G.nodes)
idx_prev=idx_orig

s_shell_dict={}

for j in idx_orig:
    s_shell_dict[j]=0
    
s_step=1
S=range(1,20000,s_step)
    
strength=np.zeros(N)

for w in range(L): #calculation of the strength for each node- weigth of a link = number of hyperedges in which the link appears
    for i in data[w]:
        strength[idx_orig.index(i)]=strength[idx_orig.index(i)]+len(data[w])-1

for y in tqdm(range(len(S))):
    s= S[y]
    idx_n_remove = [idx_prev[i] for i in range(len(idx_prev)) if strength[idx_orig.index(idx_prev[i])]<s]
    idx_core = [idx_prev[i] for i in range(len(idx_prev)) if strength[idx_orig.index(idx_prev[i])]>=s]
    
    while len(idx_n_remove)>0:
        for node_id in idx_n_remove:
            for edge_id in G.nodes.memberships(node_id):
                edge=G.edges.members(edge_id)
                for i in edge:
                    strength[idx_orig.index(i)]=strength[idx_orig.index(i)]-1 #remove link weigths
        
        idx_n_remove = [idx_core[i] for i in range(len(idx_core)) if strength[idx_orig.index(idx_core[i])]<s]
        idx_core = [idx_core[i] for i in range(len(idx_core)) if strength[idx_orig.index(idx_core[i])]>=s]

    idx_removed = list(sorted(set(idx_prev) - set(idx_core)))
    idx_prev=idx_core
    for j in idx_removed:
        s_shell_dict[j]=s-s_step
    D = len(idx_core)
    if D==0:
        break #stop the procedure when the s-core is empty

pk.dump(s_shell_dict, open('S-shell_dict_{0}_proj.pck'.format(dataset),'wb')) #s-coreness